In [ ]:
# Core Python utilities
import os
import glob
import random
import json
import shutil
import hashlib

# Data handling
import numpy as np
import pandas as pd

# Image handling
from PIL import Image, UnidentifiedImageError
import cv2

# Plotting & visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning / Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import tensorflow_hub as hub
import tensorflow_addons as tfa

# Data augmentation
import albumentations as A

# Model evaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from imblearn.over_sampling import RandomOverSampler

# Progress bars
from tqdm import tqdm

# Optional: Deployment / demo
import streamlit as st

# Optional: ONNX export for lightweight inference
import onnx
import onnxruntime



In [2]:
pip install datasets


     ---------------------------------------- 0.0/60.9 kB ? eta -:--:--
     ---------------------------------------- 60.9/60.9 kB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 0.0/57.7 kB ? eta -:--:--
     ---------------------------------------- 57.7/57.7 kB 3.2 MB/s eta 0:00:00
     ---------------------------------------- 0.0/76.3 kB ? eta -:--:--
     ---------------------------------------- 76.3/76.3 kB 4.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/494.8 kB ? eta -:--:--
   -------------- ------------------------- 184.3/494.8 kB 5.6 MB/s eta 0:00:01
   ------------------------------- -------- 389.1/494.8 kB 4.9 MB/s eta 0:00:01
   ---------------------------------------- 494.8/494.8 kB 3.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/116.3 kB ? eta -:--:--
   ---------------------------------------- 116.3/116.3 kB 3.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/193.6 kB ? eta -:--:--
   --------


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("QCRI/CrisisMMD")

# Access specific splits
train_data = dataset["train"]
dev_data = dataset["dev"]
test_data = dataset["test"]


README.md: 0.00B [00:00, ?B/s]

C:\Users\Hello\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Hello\.cache\huggingface\hub\datasets--QCRI--CrisisMMD. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


ValueError: Config name is missing.
Please pick one among the available configs: ['humanitarian', 'informative', 'damage']
Example of usage:
	`load_dataset('QCRI/CrisisMMD', 'humanitarian')`